In [1]:
!pip install -q autogen gradio

import os
from autogen import AssistantAgent
from autogen import UserProxyAgent

import google.generativeai as genai
from google.colab import userdata

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 826.8/826.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.4 MB/s eta 0:00:00


In [23]:
# Carregando chave da API Gemini salva no Colab
GOOGLE_API_KEY = userdata.get("chave_api")
genai.configure(api_key=GOOGLE_API_KEY)

# Criando o agente assistente com Gemini
assistant = AssistantAgent(
    name="ORIENS",
    llm_config={
        "config_list": [
            {
                "model": "gemini-2.0-flash",
                "api_key": GOOGLE_API_KEY,
                "base_url": "https://generativelanguage.googleapis.com/v1beta",
                "api_type": "google",
            }
        ]
    },
    system_message="""
    "Você é um analista de dados experiente e proativo, responsável por explorar e responder perguntas relacionadas a arquivos contidos em um arquivo 202401_NFs.zip localizado na pasta content.

Esse 202401_NFs.zip contém arquivos que, uma vez extraídos e carregados, geram dois principais: 202401_NFs_Cabecalho.csv e 202401_NFs_Itens.csv.

Descrição esperada dos dados:

202401_NFs_Cabecalho.csv: contém dados agregados de pedidos ou transações.

202401_NFs_Itens.csv: contém a lista de produtos ou serviços associados a cada pedido.

Suas responsabilidades incluem:

Ler o arquivo 202401_NFs.zip diretamente da pasta content, extraindo e carregando os arquivos relevantes.

Inspecionar, limpar e preparar os dados se necessário (tratamento de nulos, tipos de dados, normalização, etc.).

Utilizar Python e bibliotecas como pandas, numpy, matplotlib, seaborn, ou outras adequadas ao problema.

Apresentar os resultados de forma clara e objetiva, incluindo:

Código limpo e bem comentado;

Tabelas ou gráficos simples e informativos, quando relevantes;

Resumo interpretativo com os principais insights.

Evite jargões desnecessários e mantenha uma linguagem acessível. Foque em entregar análises úteis, diretas e acionáveis com base no conteúdo dos arquivos. responder às perguntas de forma clara, **direta** e **resumida**, como em uma conversa estilo chatbot.

Sempre que possível, **responda à pergunta de forma clara**.
Evite explicar o que vai fazer a menos que o usuário peça.

Raciocine mas não mostre seu raciocínio ao usuário, apenas a **resposta final**. Se não souber, diga claramente"""
)

In [24]:
user = UserProxyAgent(
    name="Usuário",
    human_input_mode="ALWAYS",
    max_consecutive_auto_reply=3,
    is_termination_msg=lambda x: x.get("content", "").strip().lower() == "sair",
    code_execution_config={"use_docker": False},
    system_message="Execute o código fornecido e relate o resultado. Responda 'sair' para encerrar.",
)

In [25]:
assistant.enable_print = False
user.enable_print = False

In [26]:
class SimpleChatManager:
    def __init__(self, assistant_agent):
        self.assistant = assistant_agent
        self.history = []

    def ask(self, question):
        self.history.append({"role": "user", "content": question})
        reply = self.assistant.generate_reply(messages=self.history)
        if isinstance(reply, dict):
            resposta = reply.get("content", "").strip()
            self.history.append({"role": "assistant", "content": resposta})
            return resposta
        else:
            return "Erro ao gerar resposta."

chat_manager = SimpleChatManager(assistant)

In [21]:
import gradio as gr

In [27]:
def responder(pergunta):
    if pergunta.strip().lower() in ["sair", "exit", "fechar"]:
        return "👋 Encerrando a sessão. Até a próxima!"
    resposta = chat_manager.ask(pergunta)
    return resposta

iface = gr.Interface(
    fn=responder,
    inputs=gr.Textbox(label="Pergunte algo ao ORIENS"),
    outputs=gr.Textbox(label="Resposta do ORIENS"),
    title="💬 Chat com ORIENS - Agente de Dados",
    description="Faça perguntas sobre os arquivos de notas fiscais (202401_NFs.zip) disponíveis na pasta /content.",
    theme="soft",
)

iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b5f61f3e49e028c58c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
